In [1]:
import pandas as pd
import itertools
import numpy as np
import gc

In [2]:
csv= '/home/mei/nas/docker/thesis/data/csv/'
hdf= '/home/mei/nas/docker/thesis/data/hdf/'

In [5]:
def round_up(x, base=5):
    return base * round(x/base)

In [4]:
print('==> Loading data from timeseries files...')
timeseries_lab = pd.read_csv(csv + 'timeserieslab.csv', low_memory=False)
timeseries_periodic = pd.read_csv(csv+ 'timeseriesperiodic.csv')

==> Loading data from timeseries files...


In [6]:
print("there are {} patients in the  and {} records in lab table.".format(len(list(timeseries_lab ['patientunitstayid'].unique())),len(timeseries_lab)))
print("there are {} patients in the  and {} records in vital periodic table.".format(len(list(timeseries_periodic ['patientunitstayid'].unique())),len(timeseries_periodic)))

there are 12260 patients in the  and 2337787 records in lab table.
there are 12260 patients in the  and 10671165 records in vital periodic table.


In [7]:
timeseries_lab.set_index(['patientunitstayid','labresultoffset'], inplace=True)
timeseries_periodic.set_index(['patientunitstayid','observationoffset'], inplace=True)
timeseries_lab.rename(round_up, level = 'labresultoffset', inplace = True)
timeseries_periodic.rename(round_up, level = 'observationoffset', inplace = True)
timeseries_lab.sort_index(inplace=True)
timeseries_periodic.sort_index(inplace=True)

In [8]:
def reconfigure_timeseries(timeseries, offset_column, feature_column=None, test=False):
    """
    Reconfigure timeseries data by setting multi-index and pivoting if necessary.
    """
    timeseries.reset_index(inplace=True)
    if test:
        timeseries = timeseries.iloc[:5000]  # Limit for testing
    timeseries.set_index(['patientunitstayid', pd.to_timedelta(timeseries[offset_column], unit='min')], inplace=True)
    timeseries.drop(columns=offset_column, inplace=True)
    if feature_column:
        timeseries = timeseries.pivot_table(columns=feature_column, index=timeseries.index)
    timeseries.index = pd.MultiIndex.from_tuples(timeseries.index, names=['patient', 'time'])
    return timeseries

In [9]:
test=False
print('==> Reconfiguring lab timeseries...')
lab = reconfigure_timeseries(timeseries_lab, 'labresultoffset', 'labname', test)
lab.columns=lab.columns.droplevel()

print('==> Reconfiguring periodic timeseries...')
periodic = reconfigure_timeseries(timeseries_periodic, 'observationoffset', test)

==> Reconfiguring lab timeseries...
==> Reconfiguring periodic timeseries...


In [3]:
flat=pd.read_csv(csv + 'preprocessed_flat_drug.csv')
labels = pd.read_csv(csv + 'preprocessed_labels.csv')
diagnoses= pd.read_csv(csv + 'preprocessed_diagnoses.csv')

In [4]:
common_id = list(set(flat['patient']).intersection(set(labels['patient']).intersection(set(diagnoses['patient']))))

In [5]:
len(common_id)

11698

In [13]:
lab = lab.reset_index()
lab = lab[lab['patient'].isin(common_id)]
# lab['time'] = pd.to_timedelta(lab['time'])
lab = lab[lab['time'] <= pd.to_timedelta('14 days')]

lab = lab.set_index(['patient', 'time'])

periodic = periodic.reset_index()
periodic = periodic[periodic['patient'].isin(common_id)]
periodic = periodic.set_index(['patient', 'time'])

In [14]:
print('==> Combining data together...')
merged = pd.concat([lab, periodic], axis=0, sort=False)

==> Combining data together...


In [15]:
possible_value_ranges = {
   "temperature": (32,43),        # Temperature
    "sao2": (40, 100),             # SpO2
    "heartrate": (30, 400),        # Heart rate
    "respiration": (0, 60),        # Resp. rate
    "cvp": (0, 20),                # CVP
    
    # "etco2": (0, 60),             # EtCO2
    "systemicsystolic": (40, 300), # BP systolic
    "systemicdiastolic": (20, 150),# BP diastolic
    "systemicmean": (30, 200),     # BP mean
}

In [16]:
def filter_vital_signs(data, ranges):
    for column, (min_val, max_val) in ranges.items():
        if column in data.columns:
            data = data[(data[column].isna()) | ((data[column] >= min_val) & (data[column] <= max_val))]
    return data

merged = filter_vital_signs(merged, possible_value_ranges)

# 计算 0.1% 和 99.9% 分位数
low_quantile = merged.quantile(0.001, numeric_only=True)  # 0.1% 分位数
high_quantile= merged.quantile(0.999, numeric_only=True)  # 99.9% 分位数

# 仅保留在 [0.1%, 99.9%] 之间的值
merged = merged[(merged >= low_quantile) & (merged <= high_quantile)]

print("select valid vlaue of vital signs")
print("There are {} patients and {} records in the vital periodic table.".format(
    merged.index.get_level_values('patient').nunique(),
    len(merged)
))


select valid vlaue of vital signs
There are 11698 patients and 10237638 records in the vital periodic table.


In [16]:
# merged.to_csv(csv + 'preprocessed_merged.csv')
# merged=pd.read_csv(csv + 'preprocessed_merged.csv').set_index(['patient', 'time'])

In [17]:
merged

-bands  -basos  -eos  -lymphs  -monos  -polys  \
patient time                                                             
252784  0 days 00:20:00     NaN     NaN   NaN      NaN     NaN     NaN   
        0 days 00:40:00     NaN     NaN   NaN      NaN     NaN     NaN   
        0 days 01:20:00     NaN     NaN   NaN      NaN     NaN     NaN   
        0 days 01:40:00     NaN     NaN   NaN      NaN     NaN     NaN   
        0 days 02:35:00     NaN     NaN   NaN      NaN     NaN     NaN   
...                         ...     ...   ...      ...     ...     ...   
3348105 2 days 19:25:00     NaN     NaN   NaN      NaN     NaN     NaN   
        2 days 19:30:00     NaN     NaN   NaN      NaN     NaN     NaN   
        2 days 19:35:00     NaN     NaN   NaN      NaN     NaN     NaN   
        2 days 19:40:00     NaN     NaN   NaN      NaN     NaN     NaN   
        2 days 19:45:00     NaN     NaN   NaN      NaN     NaN     NaN   

                         24 h urine protein  24 h urine urea nitrogen  \
patient time                                                            
252784  0 days 00:20:00                 NaN                       NaN   
        0 days 00:40:00                 NaN                       NaN   
        0 days 01:20:00                 NaN                       NaN   
        0 days 01:40:00                 NaN                       NaN   
        0 days 02:35:00                 NaN                       NaN   
...                                     ...                       ...   
3348105 2 days 19:25:00                 NaN                       NaN   
        2 days 19:30:00                 NaN                       NaN   
        2 days 19:35:00                 NaN                       NaN   
        2 days 19:40:00                 NaN                       NaN   
        2 days 19:45:00                 NaN                       NaN   

                         ALT (SGPT)  ANF/ANA  ...  sao2  heartrate  \
patient time                                  ...                    
252784  0 days 00:20:00         NaN      NaN  ...   NaN        NaN   
        0 days 00:40:00         NaN      NaN  ...   NaN        NaN   
        0 days 01:20:00         NaN      NaN  ...   NaN        NaN   
        0 days 01:40:00         NaN      NaN  ...   NaN        NaN   
        0 days 02:35:00         NaN      NaN  ...   NaN        NaN   
...                             ...      ...  ...   ...        ...   
3348105 2 days 19:25:00         NaN      NaN  ...   NaN       85.0   
        2 days 19:30:00         NaN      NaN  ...   NaN       86.0   
        2 days 19:35:00         NaN      NaN  ...   NaN       97.0   
        2 days 19:40:00         NaN      NaN  ...   NaN       97.0   
        2 days 19:45:00         NaN      NaN  ...   NaN      113.0   

                         respiration  cvp  systemicsystolic  \
patient time                                                  
252784  0 days 00:20:00          NaN  NaN               NaN   
        0 days 00:40:00          NaN  NaN               NaN   
        0 days 01:20:00          NaN  NaN               NaN   
        0 days 01:40:00          NaN  NaN               NaN   
        0 days 02:35:00          NaN  NaN               NaN   
...                              ...  ...               ...   
3348105 2 days 19:25:00          NaN  NaN               NaN   
        2 days 19:30:00          NaN  NaN               NaN   
        2 days 19:35:00          NaN  NaN               NaN   
        2 days 19:40:00          NaN  NaN               NaN   
        2 days 19:45:00          NaN  NaN               NaN   

                         systemicdiastolic  systemicmean  st1  st2  st3  
patient time                                                             
252784  0 days 00:20:00                NaN           NaN  NaN  NaN  NaN  
        0 days 00:40:00                NaN           NaN  NaN  NaN  NaN  
        0 days 01:20:00                NaN           NaN  NaN  NaN  NaN  
        0 d

In [18]:
def gen_patient_chunk(patients, merged, size=500): # 500 is a good size for LSTM
    """
    Generate patient data chunks for processing.
    """
    it = iter(patients)
    chunk = list(itertools.islice(it, size))
    while chunk:
        yield merged.loc[chunk]
        chunk = list(itertools.islice(it, size))

In [31]:
def resample(timeseries):
    resampled_data = []
    for patient, group in timeseries.groupby(level=0):  # process each patient separately
            # **1.resample **
            group = group.droplevel(0)  # remove patient from index
            group.index = group.index.ceil(freq='5min')  # round up to nearest 5 minutes
            resampled = group.resample('5min', closed='right', label='right').mean()  # resample to 5 minutes
            resampled['patient'] = patient  # add patient back to the DataFrame
            resampled = resampled.ffill()  # fill missing values with the last known value
            resampled.fillna(0, inplace=True) # fill remaining missing values with 0
            
            # **2.reset the multi index**
            resampled.reset_index(inplace=True)  # remove the MultiIndex
            resampled['time'] = range(1, len(resampled) + 1) 
            resampled.set_index(['patient', 'time'], inplace=True)  # set new MultiIndex
            
            resampled_data.append(resampled)
 
    final = pd.concat(resampled_data)
    return final

In [32]:
patients = merged.index.unique(level=0)
gen_chunks = gen_patient_chunk(patients, merged)
header = True
print('==> Initiating main processing loop...')

for i, patient_chunk in enumerate(gen_chunks, start=1):
    final = resample(patient_chunk)
    final.to_hdf(hdf + "final_timeseries.h5", key="df", mode="a", complevel=5, complib="zlib", format="table", append=True)

    print(f'==> Processed {i * 500} patients...')
    
    del patient_chunk # free up memory
    gc.collect()

==> Initiating main processing loop...
==> Processed 500 patients...
==> Processed 1000 patients...
==> Processed 1500 patients...
==> Processed 2000 patients...
==> Processed 2500 patients...
==> Processed 3000 patients...
==> Processed 3500 patients...
==> Processed 4000 patients...
==> Processed 4500 patients...
==> Processed 5000 patients...
==> Processed 5500 patients...
==> Processed 6000 patients...
==> Processed 6500 patients...
==> Processed 7000 patients...
==> Processed 7500 patients...
==> Processed 8000 patients...
==> Processed 8500 patients...
==> Processed 9000 patients...
==> Processed 9500 patients...
==> Processed 10000 patients...
==> Processed 10500 patients...
==> Processed 11000 patients...
==> Processed 11500 patients...
==> Processed 12000 patients...


In [35]:
merged

-bands  -basos  -eos  -lymphs  -monos  -polys  \
patient time                                                             
252784  0 days 00:20:00     NaN     NaN   NaN      NaN     NaN     NaN   
        0 days 00:40:00     NaN     NaN   NaN      NaN     NaN     NaN   
        0 days 01:20:00     NaN     NaN   NaN      NaN     NaN     NaN   
        0 days 01:40:00     NaN     NaN   NaN      NaN     NaN     NaN   
        0 days 02:35:00     NaN     NaN   NaN      NaN     NaN     NaN   
...                         ...     ...   ...      ...     ...     ...   
3348105 2 days 19:25:00     NaN     NaN   NaN      NaN     NaN     NaN   
        2 days 19:30:00     NaN     NaN   NaN      NaN     NaN     NaN   
        2 days 19:35:00     NaN     NaN   NaN      NaN     NaN     NaN   
        2 days 19:40:00     NaN     NaN   NaN      NaN     NaN     NaN   
        2 days 19:45:00     NaN     NaN   NaN      NaN     NaN     NaN   

                         24 h urine protein  24 h urine urea nitrogen  \
patient time                                                            
252784  0 days 00:20:00                 NaN                       NaN   
        0 days 00:40:00                 NaN                       NaN   
        0 days 01:20:00                 NaN                       NaN   
        0 days 01:40:00                 NaN                       NaN   
        0 days 02:35:00                 NaN                       NaN   
...                                     ...                       ...   
3348105 2 days 19:25:00                 NaN                       NaN   
        2 days 19:30:00                 NaN                       NaN   
        2 days 19:35:00                 NaN                       NaN   
        2 days 19:40:00                 NaN                       NaN   
        2 days 19:45:00                 NaN                       NaN   

                         ALT (SGPT)  ANF/ANA  ...  sao2  heartrate  \
patient time                                  ...                    
252784  0 days 00:20:00         NaN      NaN  ...   NaN        NaN   
        0 days 00:40:00         NaN      NaN  ...   NaN        NaN   
        0 days 01:20:00         NaN      NaN  ...   NaN        NaN   
        0 days 01:40:00         NaN      NaN  ...   NaN        NaN   
        0 days 02:35:00         NaN      NaN  ...   NaN        NaN   
...                             ...      ...  ...   ...        ...   
3348105 2 days 19:25:00         NaN      NaN  ...   NaN       85.0   
        2 days 19:30:00         NaN      NaN  ...   NaN       86.0   
        2 days 19:35:00         NaN      NaN  ...   NaN       97.0   
        2 days 19:40:00         NaN      NaN  ...   NaN       97.0   
        2 days 19:45:00         NaN      NaN  ...   NaN      113.0   

                         respiration  cvp  systemicsystolic  \
patient time                                                  
252784  0 days 00:20:00          NaN  NaN               NaN   
        0 days 00:40:00          NaN  NaN               NaN   
        0 days 01:20:00          NaN  NaN               NaN   
        0 days 01:40:00          NaN  NaN               NaN   
        0 days 02:35:00          NaN  NaN               NaN   
...                              ...  ...               ...   
3348105 2 days 19:25:00          NaN  NaN               NaN   
        2 days 19:30:00          NaN  NaN               NaN   
        2 days 19:35:00          NaN  NaN               NaN   
        2 days 19:40:00          NaN  NaN               NaN   
        2 days 19:45:00          NaN  NaN               NaN   

                         systemicdiastolic  systemicmean  st1  st2  st3  
patient time                                                             
252784  0 days 00:20:00                NaN           NaN  NaN  NaN  NaN  
        0 days 00:40:00                NaN           NaN  NaN  NaN  NaN  
        0 days 01:20:00                NaN           NaN  NaN  NaN  NaN  
        0 d

In [3]:
final_ts= pd.read_hdf(hdf+ "final_timeseries.h5", key="df") # key is the name of the key in the hdf file

In [8]:
final_ts

-bands  -basos  -eos  -lymphs  -monos  -polys  \
patient time                                                  
252784  1        0.0     0.0   0.0      0.0     0.0     0.0   
        2        0.0     0.0   0.0      0.0     0.0     0.0   
        3        0.0     0.0   0.0      0.0     0.0     0.0   
        4        0.0     0.0   0.0      0.0     0.0     0.0   
        5        0.0     0.0   0.0      0.0     0.0     0.0   
...              ...     ...   ...      ...     ...     ...   
3348105 1500     0.0     1.0   2.0     12.0    13.0     0.0   
        1501     0.0     1.0   2.0     12.0    13.0     0.0   
        1502     0.0     1.0   2.0     12.0    13.0     0.0   
        1503     0.0     1.0   2.0     12.0    13.0     0.0   
        1504     0.0     1.0   2.0     12.0    13.0     0.0   

              24 h urine protein  24 h urine urea nitrogen  ALT (SGPT)  \
patient time                                                             
252784  1                    0.0                       0.0         0.0   
        2                    0.0                       0.0         0.0   
        3                    0.0                       0.0         0.0   
        4                    0.0                       0.0         0.0   
        5                    0.0                       0.0         0.0   
...                          ...                       ...         ...   
3348105 1500                 0.0                       0.0        17.0   
        1501                 0.0                       0.0        17.0   
        1502                 0.0                       0.0        17.0   
        1503                 0.0                       0.0        17.0   
        1504                 0.0                       0.0        17.0   

              ANF/ANA  ...   sao2  heartrate  respiration  cvp  \
patient time           ...                                       
252784  1         0.0  ...  100.0      106.0         22.0  0.0   
        2         0.0  ...  100.0      107.0         24.0  0.0   
        3         0.0  ...  100.0      109.0         21.0  0.0   
        4         0.0  ...  100.0      110.0         26.0  0.0   
        5         0.0  ...   98.0      116.0         26.0  0.0   
...               ...  ...    ...        ...          ...  ...   
3348105 1500      0.0  ...   98.0      113.0          9.0  0.0   
        1501      0.0  ...   98.0      113.0          9.0  0.0   
        1502      0.0  ...   98.0      113.0          9.0  0.0   
        1503      0.0  ...   98.0      113.0          9.0  0.0   
        1504      0.0  ...   98.0      113.0          9.0  0.0   

              systemicsystolic  systemicdiastolic  systemicmean   st1   st2  \
patient time                                                                  
252784  1                  0.0                0.0           0.0  0.05  0.05   
        2                  0.0                0.0           0.0  0.10  0.00   
        3                  0.0                0.0           0.0  0.00  0.10   
        4                  0.0                0.0           0.0  0.10  0.00   
        5                  0.0                0.0           0.0  0.00  0.00   
...                        ...                ...           ...   ...   ...   
3348105 1500               0.0                0.0           0.0  0.00  0.00   
        1501               0.0                0.0           0.0  0.00  0.00   
        1502               0.0                0.0           0.0  0.00  0.00   
        1503               0.0                0.0           0.0  0.00  0.00   
        1504               0.0                0.0           0.0  0.00  0.00   

              st3  
patient time       
252784  1     0.0  
        2    -0.1  
        3     0.0  
        4    -0.1  
        5     0.0  
...           ...  
3348105 1500  0.0  
        1501  0.0  
        1502  0.0  
        1503  0.0  
        1504  0.0  

[18094141 rows x 162 columns]